In [30]:
import os
import sys
import pickle
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch as t
sys.path.append(os.path.dirname(os.path.abspath('.')))
import configs
from datasets import TruthfulQADataset, DishonestQADataset, AmongUsDataset, RolePlayingDataset, RepEngDataset
from probes import LinearProbe

from configs import config_phi4

In [31]:
dataset_name = 'RepEngDataset'
config = config_phi4
model, tokenizer, device = None, None, 'cpu'
EXPT_NAME: str = "2025-02-01_phi_phi_100_games_v3"
# dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, expt_name=EXPT_NAME, test_split=0)
dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, test_split=0.2)

In [32]:
# dataset.populate_dataset(force_redo=False, just_load=True)
dataset.populate_dataset(force_redo=False)

Loading existing chunk from ./data/RepEng_phi4_acts/chunk_0.pkl


In [33]:
if dataset_name == 'AmongUsDataset':
    # Get original train loader
    train_loader = dataset.get_train(batch_size=32, num_tokens=10, chunk_idx=0)

    # Create new dataset with flipped labels (so 1 is crewmate and 0 is impostor)
    flipped_dataset = [(x, 1-y) for x,y in train_loader.dataset]

    # Create new train loader with flipped labels (won't need this fix when we recreate AmongUs activations)
    train_loader = t.utils.data.DataLoader(
        flipped_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0,
        pin_memory=True
    )

In [34]:
probe = LinearProbe(input_dim=dataset.activation_size, device=device)

print(f'Training probe on {len(train_loader)} batches and {len(train_loader.dataset)} samples.')

Training probe on 32 batches and 1000 samples.


In [35]:
probe.fit(train_loader, epochs=10)

Epoch 1: Train Loss = 0.3984, Train Acc = 0.8120
Epoch 2: Train Loss = 0.1279, Train Acc = 0.9800
Epoch 3: Train Loss = 0.0731, Train Acc = 0.9980
Epoch 4: Train Loss = 0.0474, Train Acc = 1.0000
Epoch 5: Train Loss = 0.0384, Train Acc = 0.9970
Epoch 6: Train Loss = 0.0287, Train Acc = 1.0000
Epoch 7: Train Loss = 0.0239, Train Acc = 1.0000
Epoch 8: Train Loss = 0.0201, Train Acc = 1.0000
Epoch 9: Train Loss = 0.0178, Train Acc = 1.0000
Epoch 10: Train Loss = 0.0158, Train Acc = 1.0000
Final Train Acc: 1.0000


In [36]:
checkpoint_path = f'checkpoints/{dataset_name}_probe_{config["short_name"]}.pkl'
with open(checkpoint_path, 'wb') as f:
    pickle.dump(probe, f)
    print(f"Probe saved to {checkpoint_path}")

Probe saved to checkpoints/RepEngDataset_probe_phi4.pkl
